## Tecnologias de Processamento de Dados | Fase 2
### Grupo 17 / 2019.20 / Faculdade de Ciências da Universidade de Lisboa

* **João Ferreira** / 54121 / 25 horas de trabalho
* **Mariana Santos** / 49525 / 25 horas de trabalho
* **Tiago Pereira** / 49174 / 25 horas de trabalho
* **Vasco Leitão** / 49455 / 25 horas de trabalho

--------------------------------------------------------------

### Importação dos datasets relevantes ao problema

Datasets a importar (fontes disponíveis no relatório da fase 1):
* US Accidents
* TMC's e eventos associados
* Limite máximo de velocidade por estado
* Dados de consumo de álcool por estado
* Quantidade de automóveis registados por estado
* Classificação urbana e rural para cada County

In [1]:
import numpy as np
import pandas as pd
import re
import datetime as dt
import urllib.request
from IPython.display import Image
from IPython.core.display import HTML
from collections import defaultdict, Counter

##### Importação do dataset US Accidents

Para propósitos de execução do notebook e de armazenamento vamos trabalhar com uma *sample* com 50000 registos.

In [2]:
def generate_sample(filename, s):
    #n = np.sum([1 for _ in open(filename)]) - 1 # number of records in file (excludes header)
    n = 2974335
    indices = np.arange(1, n + 1)
    skip = np.sort(np.random.choice(indices, size=n - s, replace=False)) # random indices to skip

    return pd.read_csv(filename, skiprows=skip)

In [3]:
us_acidents = generate_sample('https://www.dropbox.com/s/hsu5jcg4qcxui8b/US_Accidents_Dec19.csv?dl=1', 50000)

us_acidents.head(5)

# Muitos dados que temos não incluem este estado - Não é um estado
us_acidents = us_acidents[us_acidents["State"] != 'DC'].reset_index(drop=True)

##### Importação do dataset relativo a TMCs / Criação da Dimensão TMC

In [4]:
# import event codes
event_code = pd.read_csv("https://docs.google.com/uc?export=download&id=1gcxQcC65sIGEjQSJ1CfPfFWAlbeXoNHf", delimiter=';')
event_code = event_code.rename(columns={"Code": "TMC_Key", "Description":"Event"})

# add out-of-range key
event_code = event_code.append({'TMC_Key': -1, 'Event': 'unidentified'}, ignore_index=True)
event_code.head(5)

,TMC_Key,Event
0,1,traffic problem
1,2,queuing traffic (with average speeds Q). Dange...
2,11,overheight warning system triggered
3,12,"(Q) accident(s), traffic being directed around..."
4,16,"closed, rescue and recovery work in progress"


##### Importação do dataset sobre consumo de álcool

In [5]:
def discretize_rate(x):
    if x <= 2.0:
        return '(Low) rate <= 2.0'
    elif 2.0 < x <= 2.5:
        return '(Medium) 2.0 < rate <= 2.5'
    elif 2.5 < x <= 3.5:
        return '(High) 2.5 < rate <= 3.5'
    else:
        return '(Very High) rate > 3.5'

# import alcohool consumption rates by state
alcohol_consumption = pd.read_csv("https://docs.google.com/uc?export=download&id=1G9Mk-VtCNFo8wHf1qWDdgrEj5KmAhemY", delimiter=';')

alcohol_consumption.at[20, "State"] = "Massachusetts"

# sort and reset index
alcohol_consumption = alcohol_consumption.sort_values(by=['rate'], ascending=False).reset_index(drop=True)

alcohol_consumption['DiscreteRate'] = alcohol_consumption['rate'].map(discretize_rate)

alcohol_dict = pd.Series(alcohol_consumption['DiscreteRate'].values, index=alcohol_consumption['State']).to_dict()
alcohol_dict

{'New Hampshire': '(Very High) rate > 3.5',
 'Delaware': '(Very High) rate > 3.5',
 'Nevada': '(High) 2.5 < rate <= 3.5',
 'North Dakota': '(High) 2.5 < rate <= 3.5',
 'Montana': '(High) 2.5 < rate <= 3.5',
 'Vermont': '(High) 2.5 < rate <= 3.5',
 'Wisconsin': '(High) 2.5 < rate <= 3.5',
 'Idaho': '(High) 2.5 < rate <= 3.5',
 'South Dakota': '(High) 2.5 < rate <= 3.5',
 'Colorado': '(High) 2.5 < rate <= 3.5',
 'Maine': '(High) 2.5 < rate <= 3.5',
 'Alaska': '(High) 2.5 < rate <= 3.5',
 'Minnesota': '(High) 2.5 < rate <= 3.5',
 'Oregon': '(High) 2.5 < rate <= 3.5',
 'Wyoming': '(High) 2.5 < rate <= 3.5',
 'Hawaii': '(High) 2.5 < rate <= 3.5',
 'Florida': '(High) 2.5 < rate <= 3.5',
 'Rhode Island': '(High) 2.5 < rate <= 3.5',
 'Massachusetts': '(High) 2.5 < rate <= 3.5',
 'Louisiana': '(High) 2.5 < rate <= 3.5',
 'Missouri': '(High) 2.5 < rate <= 3.5',
 'Connecticut': '(Medium) 2.0 < rate <= 2.5',
 'Iowa': '(Medium) 2.0 < rate <= 2.5',
 'Illinois': '(Medium) 2.0 < rate <= 2.5',
 'Texas'

##### Importação do dataset sobre registos automóveis

In [6]:
def discretize_total(x):
    if x <= 5000000:
        return '(Low) registrations <= 5000000'
    elif 5000000 < x <= 10000000:
        return '(Medium) 5000000 < registrations <= 10000000'
    elif 10000000 < x <= 20000000:
        return '(High) 10000000 < registrations <= 20000000'
    else:
        return '(Very High) registrations > 20000000'

# import vehicle registrations by state
vehicle_registration = pd.read_csv("https://docs.google.com/uc?export=download&id=11fq6PjFn7RdjvGyTaEHvqwkKwW_zdmWe", delimiter=';')
# filter out non States
vehicle_registration = vehicle_registration[vehicle_registration['State'] != 'Dist. of Col.']

print(vehicle_registration)

vehicle_registration['Total'] = vehicle_registration['Total'].map(lambda x: int(re.sub(' ', '', x)))

# sort and reset index
vehicle_registration = vehicle_registration.sort_values(by=['Total'], ascending=False).reset_index(drop=True)

vehicle_registration['DiscreteTotal'] = vehicle_registration['Total'].map(discretize_total)

vehicle_dict = pd.Series(vehicle_registration['DiscreteTotal'].values, index=vehicle_registration['State']).to_dict()
vehicle_dict

             State        Total
0          Alabama   5 300 199 
1           Alaska     803 684 
2          Arizona   5 806 313 
3         Arkansas   2 817 145 
4       California  31 022 328 
5         Colorado   5 356 018 
6      Connecticut   2 879 802 
7         Delaware     1008468 
9          Florida    17496002 
10         Georgia     8512550 
11          Hawaii     1267385 
12           Idaho     1879670 
13        Illinois    10588910 
14         Indiana     6190736 
15            Iowa     3691892 
16          Kansas     2684010 
17        Kentucky     4368285 
18       Louisiana     3885119 
19           Maine     1125588 
20        Maryland     4204846 
21   Massachusetts     5061499 
22        Michigan     8386831 
23       Minnesota     5404277 
24     Mississippi     2067498 
25        Missouri     5498675 
26         Montana     1845338 
27        Nebraska     1961309 
28          Nevada     2514338 
29   New Hampshire     1346318 
30      New Jersey     6055389 
31      

{'California': '(Very High) registrations > 20000000',
 'Texas': '(Very High) registrations > 20000000',
 'Florida': '(High) 10000000 < registrations <= 20000000',
 'New York': '(High) 10000000 < registrations <= 20000000',
 'Ohio': '(High) 10000000 < registrations <= 20000000',
 'Pennsylvania': '(High) 10000000 < registrations <= 20000000',
 'Illinois': '(High) 10000000 < registrations <= 20000000',
 'Georgia': '(Medium) 5000000 < registrations <= 10000000',
 'Michigan': '(Medium) 5000000 < registrations <= 10000000',
 'North Carolina': '(Medium) 5000000 < registrations <= 10000000',
 'Virginia': '(Medium) 5000000 < registrations <= 10000000',
 'Washington': '(Medium) 5000000 < registrations <= 10000000',
 'Indiana': '(Medium) 5000000 < registrations <= 10000000',
 'New Jersey': '(Medium) 5000000 < registrations <= 10000000',
 'Arizona': '(Medium) 5000000 < registrations <= 10000000',
 'Tennessee': '(Medium) 5000000 < registrations <= 10000000',
 'Wisconsin': '(Medium) 5000000 < regis

##### Importação do dataset sobre nível de urbanização

In [7]:
# import rural/urban information about counties
urban = pd.read_excel('https://docs.google.com/uc?export=download&id=1Ccn8-oZZ5HLjwZdK3gWIdVcnCswBYm_l')

# drop unwanted information
urban = urban.drop(['State Abr.','CBSA title', 'CBSA 2012 pop', 'County 2012 pop'], axis=1)

# remove County from name
urban['County name'] = urban['County name'].map(lambda x: re.sub(' County', '', x))

urban_term = {1:'Large Central Metro',
              2:'Large Fringe Metro',
              3:'Medium Metro',
              4:'Small Metro',
              5:'Micropolitan',
              6:'Non-Core'}

urban["2013 code"] = urban["2013 code"].map(urban_term)

urban_dict = pd.Series(urban['2013 code'].values, index=urban['County name']).to_dict()
urban_dict

{'Autauga': 'Medium Metro',
 'Baldwin': 'Micropolitan',
 'Barbour': 'Non-Core',
 'Bibb': 'Small Metro',
 'Blount': 'Medium Metro',
 'Bullock': 'Non-Core',
 'Butler': 'Large Fringe Metro',
 'Calhoun': 'Non-Core',
 'Chambers': 'Large Fringe Metro',
 'Cherokee': 'Micropolitan',
 'Chilton': 'Large Fringe Metro',
 'Choctaw': 'Non-Core',
 'Clarke': 'Large Fringe Metro',
 'Clay': 'Small Metro',
 'Cleburne': 'Non-Core',
 'Coffee': 'Micropolitan',
 'Colbert': 'Small Metro',
 'Conecuh': 'Non-Core',
 'Coosa': 'Micropolitan',
 'Covington': 'Non-Core',
 'Crenshaw': 'Non-Core',
 'Cullman': 'Micropolitan',
 'Dale': 'Micropolitan',
 'Dallas': 'Large Central Metro',
 'DeKalb': 'Non-Core',
 'Elmore': 'Micropolitan',
 'Escambia': 'Medium Metro',
 'Etowah': 'Small Metro',
 'Fayette': 'Small Metro',
 'Franklin': 'Medium Metro',
 'Geneva': 'Small Metro',
 'Greene': 'Small Metro',
 'Hale': 'Micropolitan',
 'Henry': 'Micropolitan',
 'Houston': 'Non-Core',
 'Jackson': 'Non-Core',
 'Jefferson': 'Micropolitan',


##### Importação do dataset sobre limites de velocidade

In [8]:
def discretize_speed(x):
    if x < 70:
        return '(Low) limit < 70 '
    elif 70 <= x < 80:
        return '(Medium) 70 <= limit < 80'
    else:
        return '(High) limit > 80'

# import maximum speed limits by state
speed_limits = pd.read_excel('https://docs.google.com/uc?export=download&id=1WPxr8k9w5fGOgL687BtCvlCHQYF0i1zV')

# drop unwanted information
speed_limits = speed_limits.drop(['Freeway (trucks)','Freeway (urban)','Divided (rural)','Undivided (rural)','Residential'], axis=1)

speed_limits = speed_limits.rename(columns={'State or territory': 'State', 'Freeway (rural)': 'Max Speed Limit (mph)'})

# List of all the US States
us_states = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado',
             'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois',
             'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
             'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana',
             'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
             'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
             'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas','Utah',
             'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']

# cleaning, avoid applying the function to missing values
speed_limits['State'] = speed_limits['State'].map(lambda x: re.sub('[^a-z A-Z]+', '', x), na_action='ignore')
speed_limits['Max Speed Limit (mph)'] = speed_limits['Max Speed Limit (mph)'].map(lambda x: re.sub('\(.*', '', x), na_action='ignore')

# filter out non US States
speed_limits = speed_limits[speed_limits['State'].isin(us_states)]

# more cleaning
speed_limits['Max Speed Limit (mph)'] = speed_limits['Max Speed Limit (mph)'].map(lambda x: int(x[:-4][-3:]))
speed_limits.at[41, 'Max Speed Limit (mph)'] = 70

# sort and reset index
speed_limits = speed_limits.sort_values(by=['Max Speed Limit (mph)'], ascending=False).reset_index(drop=True)

speed_limits['Discrete Max Speed Limit'] = speed_limits['Max Speed Limit (mph)'].map(discretize_speed)

speed_dict = pd.Series(speed_limits['Discrete Max Speed Limit'].values, index=speed_limits['State']).to_dict()
speed_dict

{'Texas': '(High) limit > 80',
 'Montana': '(High) limit > 80',
 'Idaho': '(High) limit > 80',
 'Utah': '(High) limit > 80',
 'South Dakota': '(High) limit > 80',
 'Nevada': '(High) limit > 80',
 'Wyoming': '(High) limit > 80',
 'Maine': '(Medium) 70 <= limit < 80',
 'Michigan': '(Medium) 70 <= limit < 80',
 'North Dakota': '(Medium) 70 <= limit < 80',
 'Colorado': '(Medium) 70 <= limit < 80',
 'Nebraska': '(Medium) 70 <= limit < 80',
 'Arizona': '(Medium) 70 <= limit < 80',
 'Arkansas': '(Medium) 70 <= limit < 80',
 'Kansas': '(Medium) 70 <= limit < 80',
 'Oklahoma': '(Medium) 70 <= limit < 80',
 'Louisiana': '(Medium) 70 <= limit < 80',
 'New Mexico': '(Medium) 70 <= limit < 80',
 'Ohio': '(Medium) 70 <= limit < 80',
 'Oregon': '(Medium) 70 <= limit < 80',
 'Pennsylvania': '(Medium) 70 <= limit < 80',
 'New Hampshire': '(Medium) 70 <= limit < 80',
 'South Carolina': '(Medium) 70 <= limit < 80',
 'California': '(Medium) 70 <= limit < 80',
 'Tennessee': '(Medium) 70 <= limit < 80',
 'V

### Tratamento do dataset original

#### Tratamento de dados nulos/incompletos

* As linhas que têm valores relativos ao twilight e período do dia vazios serão removidas; 
* As linhas que não têm informação relativa à cidade na qual o acidente ocorre também serão removidas;
* Os códigos TMC (Traffic Message Channel) que sejam nulos serão identificados com um número fora do domínio dos códigos existentes e não vão ser relevantes para possíveis interrogações;
* Definição de valores para dados meteorológicos nulos será feita com base na média de valores em cada atributo, ao nível da cidade, 
na respetiva semana em que ocorreu o acidente (tal será feito na criação da *dimension* Weather);
* O preenchimento de dados quem não contêm fuso horário será feito através da correspondência entre o estado onde o acidente ocorreu e o fuso horário no qual se encontra.

#### Remoção de colunas
* 'Source',
* 'Start_Lat',
* 'Country',
* 'Start_Lng',
* 'End_Lat',
* 'End_Lng',
* 'Description',
* 'Number',
* 'Side',
* 'Zipcode',
* 'Airport_Code',
* 'Weather_Timestamp',
* 'Pressure(in)', 
* 'Wind_Chill(F)'
 

In [9]:
us_acidents.columns

Index(['ID', 'Source', 'TMC', 'Severity', 'Start_Time', 'End_Time',
       'Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng', 'Distance(mi)',
       'Description', 'Number', 'Street', 'Side', 'City', 'County', 'State',
       'Zipcode', 'Country', 'Timezone', 'Airport_Code', 'Weather_Timestamp',
       'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)',
       'Visibility(mi)', 'Wind_Direction', 'Wind_Speed(mph)',
       'Precipitation(in)', 'Weather_Condition', 'Amenity', 'Bump', 'Crossing',
       'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station',
       'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop',
       'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight',
       'Astronomical_Twilight'],
      dtype='object')

In [10]:
# drop unuseful information/columns
us_acidents = us_acidents.drop(['Source', 'Start_Lat', 'Country', 'Start_Lng', 'End_Lat', 'End_Lng', 'Description', 'Number', 'Side', 'Zipcode', 'Airport_Code', 'Weather_Timestamp', 'Pressure(in)', 'Wind_Chill(F)'], axis=1)

In [11]:
# delete rows with non avaliable values
us_acidents = us_acidents.dropna(subset=['City', 'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight']).reset_index(drop=True)

In [12]:
# replace TMC values with out-of-range values, not used in OLAP.
us_acidents['TMC'].fillna(-1, inplace=True)
us_acidents['TMC'] = us_acidents['TMC'].astype(int)

In [13]:
# replace empty timezone values with the correspondent ones from each state
timezone_df = us_acidents[['State', 'Timezone']].drop_duplicates(subset="State")
timezone_dict = pd.Series(timezone_df['Timezone'].values, index=timezone_df['State']).to_dict()

us_acidents['Timezone'].fillna(us_acidents['State'].map(timezone_dict), inplace=True)

### Location Dimension

##### Inserção de Chave

In [14]:
# add location key
us_acidents.insert(us_acidents.columns.get_loc('Street'), 'LocationKey', range(1, len(us_acidents) + 1))

##### Definição dos atributos de região

In [15]:
us_acidents['State'].unique()

array(['OH', 'CA', 'FL', 'SC', 'GA', 'NE', 'IA', 'IL', 'MO', 'IN', 'MI',
       'CT', 'NY', 'MA', 'NJ', 'RI', 'KY', 'PA', 'VA', 'MD', 'DE', 'TX',
       'WA', 'NH', 'AL', 'TN', 'NC', 'MN', 'OK', 'UT', 'NM', 'LA', 'AZ',
       'OR', 'NV', 'WI', 'KS', 'CO', 'MS', 'ID', 'AR', 'VT', 'WV', 'WY',
       'ME', 'MT'], dtype=object)

In [16]:
replace_values = {'OH':'Ohio', 'WV':'West Virginia', 'CA': 'California', 'FL': 'Florida', 'GA': 'Georgia', 'SC':'South Carolina', 'NE': 'Nebraska', 'IA': 'Iowa', 'IL': 'Illinois', 'MO': 'Missouri', 'WI': 'Wisconsin',
       'IN': 'Indiana', 'MI': 'Michigan', 'NJ': 'New Jersey', 'NY': 'New York', 'CT': 'Connecticut', 'MA': 'Massachusetts', 'RI': 'Rhode Island', 'NH': 'New Hampshire', 'PA': 'Pennsylvania', 'KY': 'Kentucky', 'MD': 'Maryland',
       'VA': 'Virginia', 'DE': 'Delaware', 'TX':'Texas', 'WA': 'Washington', 'OR': 'Oregon', 'AL': 'Alabama', 'NC': 'North Carolina', 'MN': 'Minnesota', 'OK': 'Oklahoma', 'LA': 'Louisiana',
   'TN': 'Tennessee', 'UT': 'Utah', 'CO': 'Colorado', 'AZ': 'Arizona', 'NV': 'Nevada', 'KS': 'Kansas', 'MS': 'Mississippi', 'NM': 'New Mexico', 'ME': 'Maine', 'AR': 'Arkansas', 'WY': 'Wyoming','VT': 'Vermont', 'ID': 'Idaho', 'ND': 'North Dakota', 'MT': 'Montana', 'SD': 'South Dakota'}

In [17]:
us_acidents = us_acidents.replace({"State": replace_values})

In [18]:
# https://en.wikipedia.org/wiki/List_of_regions_of_the_United_States#Census_Bureau-designated_regions_and_divisions
sub_regions = {
    'New England': ['Connecticut', 'Maine', 'Massachusetts', 'New Hampshire', 'Rhode Island', 'Vermont'],
    'Mid Atlantic': ['New Jersey', 'New York','Pennsylvania'],
    'East North Central': ['Illinois', 'Indiana', 'Michigan', 'Ohio', 'Wisconsin'],
    'West North Central': ['Iowa','Kansas','Minnesota','Missouri','Nebraska','North Dakota','South Dakota'],
    'South Atlantic': ['Delaware','Florida','Georgia','Maryland','North Carolina','South Carolina','Virginia','West Virginia'],
    'East South Central': ['Alabama','Kentucky','Mississippi','Tennessee'],
    'West South Central': ['Arkansas','Louisiana','Oklahoma','Texas'],
    'Mountain': ['Arizona','Colorado','Idaho','Montana','Nevada','New Mexico','Utah','Wyoming'],
    'Pacific': ['Alaska','California','Hawaii','Oregon','Washington']
}

regions = {
    'Northeast': ['Connecticut', 'Maine', 'Massachusetts', 'New Hampshire', 'Rhode Island', 'Vermont', 'New Jersey', 'New York','Pennsylvania'],
    'Midwest': ['Illinois', 'Indiana', 'Michigan', 'Ohio', 'Wisconsin', 'Iowa','Kansas','Minnesota','Missouri','Nebraska','North Dakota','South Dakota'],
    'South': ['Delaware','Florida','Georgia','Maryland','North Carolina','South Carolina','Virginia','District of Columbia','West Virginia', 'Alabama','Kentucky','Mississippi','Tennessee', 'Arkansas','Louisiana','Oklahoma','Texas'],
    'West': ['Arizona','Colorado','Idaho','Montana','Nevada','New Mexico','Utah','Wyoming', 'Alaska','California','Hawaii','Oregon','Washington']
}

# create an inverted dictionary map
regions_dict = dict([(v, k) for k, l in regions.items() for v in l])
sub_regions_dict = dict([(v, k) for k, l in sub_regions.items() for v in l])

##### Inserção dos atributos Região e Sub-Região

In [19]:
# add subregion
us_acidents.insert(us_acidents.columns.get_loc('Timezone'), 'Subregion', us_acidents['State'].map(sub_regions_dict))
us_acidents['Subregion'].value_counts()

Pacific               13513
South Atlantic        12214
West South Central     6800
East North Central     4597
Mid Atlantic           4570
Mountain               2669
West North Central     2203
East South Central     2092
New England            1278
Name: Subregion, dtype: int64

In [20]:
# add region
us_acidents.insert(us_acidents.columns.get_loc("Timezone"), 'Region', us_acidents['State'].map(regions_dict))
us_acidents['Region'].value_counts()

South        21106
West         16182
Midwest       6800
Northeast     5848
Name: Region, dtype: int64

##### Inserção de atributo AlcoholConsumptionRate

In [21]:
# add alcohol consumption
us_acidents.insert(us_acidents.columns.get_loc('Timezone'), 'AlcoholConsumptionRate', us_acidents['State'].map(alcohol_dict))
us_acidents['AlcoholConsumptionRate'].value_counts()

(Medium) 2.0 < rate <= 2.5    36647
(High) 2.5 < rate <= 3.5       9227
(Low) rate <= 2.0              3897
(Very High) rate > 3.5          165
Name: AlcoholConsumptionRate, dtype: int64

#### Inserção do atributo NumberVehicleRegistrations

In [22]:
# add number of vehicles
us_acidents.insert(us_acidents.columns.get_loc('Timezone'), 'NumberOfVehicleRegistrations', us_acidents['State'].map(vehicle_dict))
us_acidents['NumberOfVehicleRegistrations'].value_counts()

(Very High) registrations > 20000000            16348
(Medium) 5000000 < registrations <= 10000000    14860
(High) 10000000 < registrations <= 20000000      9756
(Low) registrations <= 5000000                   8972
Name: NumberOfVehicleRegistrations, dtype: int64

#### Inserção do atributo UrbanRuralClassification

In [23]:
# add rate
us_acidents.insert(us_acidents.columns.get_loc('Timezone'), 'UrbanRuralClassification', us_acidents['County'].map(urban_dict))

# ~81000 rows dropped - Errors in County, only with manual correction allowed
us_acidents = us_acidents[us_acidents['UrbanRuralClassification'].notna()]
us_acidents.reset_index(drop=True, inplace=True)

us_acidents['UrbanRuralClassification'].value_counts()

Large Central Metro    18742
Non-Core                9188
Medium Metro            7452
Large Fringe Metro      7179
Micropolitan            3152
Small Metro             2463
Name: UrbanRuralClassification, dtype: int64

#### Inserção do atributo MaximumSpeedLimit

In [24]:
# add max speed limit
us_acidents.insert(us_acidents.columns.get_loc('Timezone'), 'MaximumSpeedLimit', us_acidents['State'].map(speed_dict))

us_acidents['MaximumSpeedLimit'].value_counts()

(Medium) 70 <= limit < 80    37961
(High) limit > 80             5942
(Low) limit < 70              4273
Name: MaximumSpeedLimit, dtype: int64

In [25]:
# create location dataframe
location_dimension = us_acidents[['LocationKey', 'Street','City','County','State','Subregion','Region','AlcoholConsumptionRate','NumberOfVehicleRegistrations','UrbanRuralClassification','MaximumSpeedLimit']]
location_dimension.columns

Index(['LocationKey', 'Street', 'City', 'County', 'State', 'Subregion',
       'Region', 'AlcoholConsumptionRate', 'NumberOfVehicleRegistrations',
       'UrbanRuralClassification', 'MaximumSpeedLimit'],
      dtype='object')

### POI Dimension

In [26]:
# add POI_Key
us_acidents.insert(us_acidents.columns.get_loc('Amenity'), 'POI_Key', range(1, len(us_acidents) + 1))

In [27]:
# create POI dataframe
poi_dimension = us_acidents.loc[:,'POI_Key':'Turning_Loop']

poi_dimension.columns

Index(['POI_Key', 'Amenity', 'Bump', 'Crossing', 'Give_Way', 'Junction',
       'No_Exit', 'Railway', 'Roundabout', 'Station', 'Stop',
       'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop'],
      dtype='object')

In [28]:
poi_dimension.head(5)

,POI_Key,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop
0,1,False,False,False,False,False,False,False,False,False,False,False,False,False
1,2,False,False,False,False,False,False,False,False,False,False,False,False,False
2,3,False,False,False,False,False,False,False,False,False,False,False,False,False
3,4,False,False,False,False,False,False,False,False,False,False,False,True,False
4,5,False,False,False,False,False,False,False,False,False,False,False,False,False


In [29]:
# replace boolean values
poi_dimension.loc[:,'Amenity':'Turning_Loop'] = poi_dimension.loc[:,'Amenity':'Turning_Loop'].replace({True: 'Present', False: 'Not Present'})

In [30]:
poi_dimension.sample(5)

,POI_Key,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop
23122,23123,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present
18656,18657,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present
43185,43186,Not Present,Not Present,Not Present,Not Present,Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present
26547,26548,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present
34099,34100,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present


### Date Dimension

##### Conversão dos registos Start_Time e End_Time para Datetime

In [31]:
# convert to timestamp datatype 
us_acidents['Start_Time'] = pd.to_datetime(us_acidents['Start_Time'])
us_acidents['End_Time'] = pd.to_datetime(us_acidents['End_Time'])

# rename columns
us_acidents = us_acidents.rename(columns={'Start_Time': 'Start_Datetime', 'End_Time': 'End_Datetime'})

##### Inserção de Chave

In [32]:
us_acidents.insert(us_acidents.columns.get_loc('Start_Datetime'), 'DateKey', range(1, len(us_acidents) + 1))

##### Inserção dos atributos StartDay, StartMonth e StartYear

In [33]:
# add starting date
us_acidents.insert(us_acidents.columns.get_loc('Start_Datetime'), 'StartDay', us_acidents['Start_Datetime'].map(lambda x: x.day))
us_acidents.insert(us_acidents.columns.get_loc('Start_Datetime'), 'StartMonth', us_acidents['Start_Datetime'].map(lambda x: x.month_name()))
us_acidents.insert(us_acidents.columns.get_loc('Start_Datetime'), 'StartYear', us_acidents['Start_Datetime'].map(lambda x: x.year))

##### Inserção dos atributos EndDay, EndMonth e EndYear

In [34]:
# add ending date
us_acidents.insert(us_acidents.columns.get_loc('Start_Datetime'), 'EndDay', us_acidents['End_Datetime'].map(lambda x: x.day)) 
us_acidents.insert(us_acidents.columns.get_loc('Start_Datetime'), 'EndMonth', us_acidents['End_Datetime'].map(lambda x: x.month_name())) 
us_acidents.insert(us_acidents.columns.get_loc('Start_Datetime'), 'EndYear', us_acidents['End_Datetime'].map(lambda x: x.year))

##### Inserção do atributo DayOfWeek

In [35]:
# add day of the week
us_acidents.insert(us_acidents.columns.get_loc('Start_Datetime'), 'DayOfWeek', us_acidents['Start_Datetime'].map(lambda x: x.day_name()))
us_acidents['DayOfWeek'].value_counts()

Tuesday      8918
Wednesday    8655
Friday       8584
Thursday     8548
Monday       8161
Saturday     2857
Sunday       2453
Name: DayOfWeek, dtype: int64

##### Inserção do atributo WeekdayWeekend

In [36]:
week_dict = {
    'Monday': 'Week Day',
    'Tuesday': 'Week Day',
    'Wednesday': 'Week Day',
    'Thursday': 'Week Day',
    'Friday': 'Week Day',
    'Saturday': 'Weekend',
    'Sunday': 'Weekend'
}

In [37]:
# add week day/weekend
us_acidents.insert(us_acidents.columns.get_loc('Start_Datetime'), 'WeekdayWeekend', us_acidents['DayOfWeek'].map(week_dict))
us_acidents[['Start_Datetime', 'DayOfWeek', 'WeekdayWeekend']].sample(5)

,Start_Datetime,DayOfWeek,WeekdayWeekend
31529,2017-12-12 16:38:54,Tuesday,Week Day
15150,2019-05-11 14:53:25,Saturday,Weekend
5236,2017-02-07 07:31:58,Tuesday,Week Day
44544,2018-06-30 23:28:45,Saturday,Weekend
35669,2017-08-15 11:50:33,Tuesday,Week Day


##### Inserção dos atributos Quarter (trimester) e WeekNumber

In [38]:
# add quarter and week number
us_acidents.insert(us_acidents.columns.get_loc('Start_Datetime'), 'Quarter', us_acidents['Start_Datetime'].map(lambda x: x.quarter))
us_acidents.insert(us_acidents.columns.get_loc('Start_Datetime'), 'WeekNumber', us_acidents['Start_Datetime'].map(lambda x: x.weekofyear))
us_acidents[['Start_Datetime','Quarter','WeekNumber']].sample(5)

,Start_Datetime,Quarter,WeekNumber
22536,2018-09-02 22:56:14,3,35
38106,2017-01-23 14:00:56,1,4
31858,2017-12-21 08:25:35,4,51
6498,2017-03-24 14:22:29,1,12
6139,2017-03-20 16:55:26,1,12


##### Inserção do atributo Holiday

In [39]:
federal_holidays = pd.read_csv('https://docs.google.com/uc?export=download&id=1fFfhfPZSVchTPvSnH4ClBP7JVGmJ0yHC', delimiter=';')
federal_holidays['date'] = pd.to_datetime(federal_holidays['date']).dt.date
federal_holidays.sample(5)

,date,holiday,exceptions
40,2019-02-09,Labor Day,
44,2019-12-24,Christmas Eve,
12,2017-02-01,New Year's Day observed,
35,2019-01-01,New Year's Day,
4,2016-04-07,Independence Day,


In [40]:
holidays_dict = pd.Series(federal_holidays['holiday'].values, index=federal_holidays['date']).to_dict()
holidays_dict

{datetime.date(2016, 1, 1): "New Year's Day",
 datetime.date(2016, 1, 18): 'Martin Luther King Jr. Day',
 datetime.date(2016, 2, 15): "Presidents' Day",
 datetime.date(2016, 5, 30): 'Memorial Day',
 datetime.date(2016, 4, 7): 'Independence Day',
 datetime.date(2016, 5, 9): 'Labor Day',
 datetime.date(2016, 10, 10): 'Columbus Day',
 datetime.date(2016, 11, 11): 'Veterans Day',
 datetime.date(2016, 11, 24): 'Thanksgiving Day',
 datetime.date(2016, 12, 25): 'Christmas Day',
 datetime.date(2016, 12, 26): 'Christmas Day observed',
 datetime.date(2017, 1, 1): "New Year's Day",
 datetime.date(2017, 2, 1): "New Year's Day observed",
 datetime.date(2017, 1, 16): 'Martin Luther King Jr. Day',
 datetime.date(2017, 2, 20): "Presidents' Day",
 datetime.date(2017, 5, 29): 'Memorial Day',
 datetime.date(2017, 4, 7): 'Independence Day',
 datetime.date(2017, 4, 9): 'Labor Day',
 datetime.date(2017, 9, 10): 'Columbus Day',
 datetime.date(2017, 10, 11): 'Veterans Day observed',
 datetime.date(2017, 11, 1

In [41]:
# add holiday
us_acidents.insert(us_acidents.columns.get_loc('Start_Datetime'), 'Holiday', us_acidents['Start_Datetime'].dt.date.map(holidays_dict))
us_acidents['Holiday'] = us_acidents['Holiday'].fillna('Not Holiday')
us_acidents['Holiday'].value_counts()

Not Holiday                   47018
Martin Luther King Jr. Day      132
Veterans Day                    131
Columbus Day                    128
Presidents' Day                 117
Labor Day                       108
Veterans Day observed            99
Thanksgiving Day                 94
Independence Day                 80
Christmas Eve                    69
Memorial Day                     64
Christmas Day                    59
New Year's Day observed          35
New Year's Day                   31
Christmas Day observed           11
Name: Holiday, dtype: int64

##### Inserção do atributo WorkDay

In [42]:
# create a boolean mask by doing element-wise logical OR
work_days_mask = (us_acidents['Holiday'] != 'Not Holiday') | (us_acidents['WeekdayWeekend'] == 'Weekend')

# add work days
us_acidents.insert(us_acidents.columns.get_loc('Start_Datetime'), 'WorkDay', work_days_mask.map({False: 'Work Day', True: 'Not Work Day'}))
us_acidents['WorkDay'].value_counts()

Work Day        41799
Not Work Day     6377
Name: WorkDay, dtype: int64

In [43]:
us_acidents[us_acidents['WorkDay'] == 'Not Work Day'][['Start_Datetime', 'DayOfWeek', 'Holiday', 'WorkDay']].sample(5)

,Start_Datetime,DayOfWeek,Holiday,WorkDay
5729,2017-03-18 16:37:26,Saturday,Not Holiday,Not Work Day
11950,2019-09-08 16:20:26,Sunday,Not Holiday,Not Work Day
22719,2018-09-09 13:45:04,Sunday,Not Holiday,Not Work Day
33489,2017-10-15 13:45:48,Sunday,Not Holiday,Not Work Day
6478,2017-04-01 20:55:05,Saturday,Not Holiday,Not Work Day


##### Inserção do atributo SchoolBreak

In [44]:
school_holidays = pd.read_csv('https://docs.google.com/uc?export=download&id=1MqMvq21Ped0QAb4t-qg_MeOWdQsVp2mN', delimiter=';')
school_holidays['Start']= pd.to_datetime(school_holidays['Start'])
school_holidays['End']= pd.to_datetime(school_holidays['End'])

In [45]:
# initialize mask
classes_mask = pd.Series([False for _ in range(len(us_acidents))])

for i in range(len(school_holidays)):
    classes_mask |= (us_acidents['Start_Datetime'] >= school_holidays.loc[i, "Start"]) & (us_acidents['Start_Datetime'] <= school_holidays.loc[i, "End"])

# add school breaks
us_acidents.insert(us_acidents.columns.get_loc('Start_Datetime'), 'SchoolBreak', classes_mask.map(lambda x: 'No Classes' if x else 'Classes'))
us_acidents['SchoolBreak'].value_counts()

Classes       38185
No Classes     9991
Name: SchoolBreak, dtype: int64

In [46]:
# create date dimension
date_dimension = us_acidents[['DateKey','StartDay','DayOfWeek','WeekdayWeekend','WeekNumber','WorkDay','Holiday','SchoolBreak','StartMonth','Quarter','StartYear','EndDay','EndMonth','EndYear']]

### Weather Dimension
* Definição de valores para dados meteorológicos nulos será feita com base na média de valores em cada atributo, ao nível da cidade, 
na respetiva semana em que ocorreu o acidente (tal será feito na criação da *dimension* Weather);

##### Inserção de chave

In [47]:
# add location key
us_acidents.insert(us_acidents.columns.get_loc('Temperature(F)'), 'WeatherKey', range(1, len(us_acidents) + 1))

##### Preenchimento de valores nulos nos atributos metereológicos

In [48]:
# Checking the NaN counts for each attribute
us_acidents[['Temperature(F)', 'Humidity(%)', 'Precipitation(in)', 'Visibility(mi)', 'Weather_Condition', 'Wind_Direction']].isna().sum()

Temperature(F)         865
Humidity(%)            908
Precipitation(in)    32185
Visibility(mi)         925
Weather_Condition      940
Wind_Direction         690
dtype: int64

In [49]:
def weather_dict_counter(df, column_name):
    """Counts weather observations in each group."""
    dic = defaultdict(Counter)
    for idx in df.index:
        w = df.loc[idx, 'WeekNumber']
        s = df.loc[idx, 'State']
        y = df.loc[idx, 'StartYear']

        v = df.loc[idx, column_name]

        if v is not np.NaN:
            dic[(y, s, w)].update([v])

    return dic

def fill_weather_disc(df, column_name):
    """Fill missing weather values (discrete) in a given column with the most common value (mode)."""
    subset = df[['StartYear', 'State', 'WeekNumber', column_name]]
    dic = weather_dict_counter(subset, column_name)
    for idx in df.index:
        if pd.isnull(df.loc[idx, column_name]):
            w = df.loc[idx, 'WeekNumber']
            s = df.loc[idx, 'State']
            y = df.loc[idx, 'StartYear']
            
            try:
                df.loc[idx, column_name] = dic[(y, s, w)].most_common(1)[0][0] # get nested value
            except:
                pass
                

def fill_weather_cont(df, column_name):
    """Fill missing weather values (continuous) in a given column with mean."""
    subset = df[['StartYear', 'State', 'WeekNumber', column_name]]
    aggr = subset.groupby(['StartYear', 'State', 'WeekNumber']).mean()
    for idx in df.index:
        if pd.isnull(df.loc[idx, column_name]):
            w = df.loc[idx, 'WeekNumber']
            s = df.loc[idx, 'State']
            y = df.loc[idx, 'StartYear']

            v = aggr.loc[(y, s, w), column_name]

            df.loc[idx, column_name] = v

In [50]:
# fill continuous values
fill_weather_cont(us_acidents, 'Temperature(F)')
fill_weather_cont(us_acidents, 'Humidity(%)')
fill_weather_cont(us_acidents, 'Precipitation(in)')
fill_weather_cont(us_acidents, 'Visibility(mi)')

# fill discrete values
fill_weather_disc(us_acidents, 'Wind_Direction')
fill_weather_disc(us_acidents, 'Weather_Condition')

In [51]:
# Check the NaN counts of each attribute after applying the NaN filling
us_acidents[['Temperature(F)', 'Humidity(%)', 'Precipitation(in)', 'Visibility(mi)', 'Weather_Condition', 'Wind_Direction']].isna().sum()

Temperature(F)          62
Humidity(%)             63
Precipitation(in)    12663
Visibility(mi)          72
Weather_Condition       75
Wind_Direction          20
dtype: int64

##### Discretização dos valores contínuos referentes a meteorologia

A discretização relativamente aos atributos da meterologia segue os seguintes moldes:
* Para os valores que são NaN (tal ocorre apenas na sample porque a função não encontra valores de referência para calcular médias e modas), a discretização vai defini-los como Unknown;
* A discretização da Humidade, Temperatura e Visibilidade foi feita de forma intuitiva, dado não existirem escalas universais que categorizem tais atributos;
* A discretização da velocidade do vento vai ter como base a **Escala de Beaufort** (fonte: https://en.wikipedia.org/wiki/Beaufort_scale);
* A discretização da intensidade de precipitação vai ter como base a categorização descrita em fontes retiradas da Wikipedia (fonte: https://en.wikipedia.org/wiki/Rain#Intensity).

In [52]:
print("Escala de Beaufort para discretização da velocidade do vento:\n")
Image(url= "https://docs.google.com/uc?export=download&id=1T2m3aTkeYsgg5vnEl79fzA8UCYWhF6BM")

Escala de Beaufort para discretização da velocidade do vento:



In [53]:
import math

def discretize_humidity(x):
    if math.isnan(x):
        return 'Unknown'
    elif x <= 25.0:
        return '(Low) rate <= 25.0'
    elif 25.0 < x <= 50.0:
        return '(Medium) 25.0 < rate <= 50.0'
    elif 50.0 < x <= 75.0:
        return '(High) 50.0 < rate <= 75.0'
    else:
        return '(Very High) rate > 75.0'

def discretize_temp(x):
    if math.isnan(x):
        return 'Unknown'
    elif x <= 32.0:
        return '(Cold) temp <= 32.0'
    elif 32.0 < x <= 50.0:
        return '(Low) 32.0 < temp <= 50.0'
    elif 50.0 < x <= 80.0:
        return '(Moderate) 50.0 < temp <= 80.0'
    elif 80.0 < x <= 100.0:
        return '(Warm) 80.0 < temp <= 100.0'
    else:
        return "(Hot) temp > 100.0"

def discretize_visibility(x):
    if math.isnan(x):
        return 'Unknown'
    elif x <= 1.0:
        return '(Low) visibility <= 1.0'
    elif 1.0 < x <= 2.0:
        return '(Reduced) 1.0 < visibility <= 2.0'
    elif 2.0 < x <= 5.0:
        return '(Moderate) 2.0 < visibility <= 5.0'
    elif 5.0 < x <= 20.0:
        return '(Good) 5.0 < visibility <= 20.0'
    elif 20.0 < x <= 50.0:
        return '(Very Good) 20.0 < visibility <= 50.0'
    else:
        return '(Excellent) visibility > 50.0'

#https://en.wikipedia.org/wiki/Beaufort_scale - converting m/s to mph
def discretize_windspeed(x):
    if math.isnan(x):
        return 'Unknown'
    elif x <= 1.12:
        return '(Calm) speed <= 1.0'
    elif 1.12 < x <= 3.36:
        return '(Light Air) 1.12 < speed <= 3.36'
    elif 3.36 < x <= 7.38:
        return '(Light Breeze) 3.36 < speed <= 7.38'
    elif 7.38 < x <= 12.30:
        return '(Gentle Breeze) 7.38 < speed <= 12.30'
    elif 12.30 < x <= 17.67:
        return '(Moderate Breeze) 12.30 < speed <= 17.67'
    elif 17.67 < x <= 23.93:
        return '(Fresh Breeze) 17.67 < speed <= 23.93'
    elif 23.93 < x <= 30.87:
        return '(Strong Breeze) 23.93 < speed <= 30.87'
    elif 30.87 < x <= 38.25:
        return '(High Wind)30.87 < speed <= 38.25'
    elif 38.25 < x <= 46.30:
        return '(Gale) 38.25 < speed <= 46.30'
    elif 46.30 < x <= 54.58:
        return '(Strong Gale) 46.30 < speed <= 54.58'
    elif 54.58 < x <= 63.53:
        return '(Storm) 54.58 < speed <= 63.53'
    elif 63.53 < x <= 72.92:
        return '(Violent Storm) 63.53 < speed <= 72.92' 
    else:
        return '(Hurricane) speed > 72.92'

# https://en.wikipedia.org/wiki/Rain#Intensity
def discretize_rainfall(x):
    if math.isnan(x):
        return 'Unknown'
    elif x <= 0.1:
        return '(Light) rate <= 0.1'
    elif 0.1 < x <= 0.39:
        return '(Moderate) 0.1 < rate <= 0.3'
    elif 0.39 < x <= 2.0:
        return '(Heavy) 0.39 < rate <= 2.0'
    else:
        return '(Violent) rate > 2.0'

us_acidents['Temperature(F)'] = us_acidents["Temperature(F)"].map(discretize_temp)
us_acidents['Humidity(%)'] = us_acidents['Humidity(%)'].map(discretize_humidity)
us_acidents['Visibility(mi)'] = us_acidents['Visibility(mi)'].map(discretize_visibility)
us_acidents['Wind_Speed(mph)'] = us_acidents['Wind_Speed(mph)'].map(discretize_windspeed)
us_acidents['Precipitation(in)'] = us_acidents['Precipitation(in)'].map(discretize_rainfall)

us_acidents.fillna({'Weather_Condition':'Unknown', 'Wind_Direction':'Unknown'}, inplace=True)


Como é possível observar, não existem quaisquer dados 'nulos' referentes à metereologia que serão carregados na BD.

In [54]:
us_acidents[['Temperature(F)', 'Humidity(%)', 'Precipitation(in)', 'Visibility(mi)', 'Weather_Condition', 'Wind_Direction']].isna().sum()

Temperature(F)       0
Humidity(%)          0
Precipitation(in)    0
Visibility(mi)       0
Weather_Condition    0
Wind_Direction       0
dtype: int64

In [55]:
## criação dimensão weather 
weather_dimension = us_acidents[['WeatherKey', 'Temperature(F)', 'Visibility(mi)', 'Humidity(%)', 'Wind_Direction', 'Wind_Speed(mph)', 'Precipitation(in)','Weather_Condition']]

### LocalTime Dimension


In [56]:
us_acidents['LocalTimeKey'] = pd.RangeIndex(start=1, stop=us_acidents.shape[0]+1)

In [58]:
us_acidents['StartTime'] = pd.DatetimeIndex(us_acidents['Start_Datetime']).time
us_acidents['EndTime'] = pd.DatetimeIndex(us_acidents['End_Datetime']).time

In [59]:
us_acidents

,ID,TMC,Severity,DateKey,StartDay,StartMonth,StartYear,EndDay,EndMonth,EndYear,...,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight,LocalTimeKey,StartTime,EndTime
0,A-23,201,2,1,8,February,2016,8,February,2016,...,False,False,False,Day,Day,Day,Day,1,11:53:19,12:23:19
1,A-64,201,2,2,9,February,2016,9,February,2016,...,False,False,False,Day,Day,Day,Day,2,08:12:44,08:42:44
2,A-197,201,2,3,16,February,2016,16,February,2016,...,False,False,False,Day,Day,Day,Day,3,10:58:52,11:28:52
3,A-235,201,2,4,18,February,2016,18,February,2016,...,False,True,False,Night,Night,Night,Night,4,05:46:52,06:16:52
4,A-477,201,3,5,3,March,2016,3,March,2016,...,False,False,False,Night,Day,Day,Day,5,18:52:00,23:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48171,A-2974084,-1,3,48172,23,August,2019,23,August,2019,...,False,False,False,Day,Day,Day,Day,48172,09:55:55,10:23:48
48172,A-2974118,-1,2,48173,23,August,2019,23,August,2019,...,False,False,False,Day,Day,Day,Day,48173,12:00:00,13:00:00
48173,A-2974209,-1,2,48174,23,August,2019,23,August,2019,...,False,False,False,Day,Day,Day,Day,48174,10:42:59,14:42:59
48174,A-2974239,-1,2,48175,23,August,2019,23,August,2019,...,False,False,False,Day,Day,Day,Day,48175,13:10:33,17:10:33


In [60]:
def define_period_of_day(start_hour, civil_tw, naut_tw, ast_tw, sunrise_sunset) :
    if (civil_tw != naut_tw) or (naut_tw != ast_tw) :
        return "Dusk"
    elif dt.time(8, 0, 0) < start_hour < dt.time(10, 0, 0) :
        return 'Morning Rush Hour'
    elif dt.time(12, 30, 0) < start_hour < dt.time(14, 30, 0) :
        return 'Lunch Time'
    elif dt.time(16, 0, 0) < start_hour < dt.time(18, 0, 0) :
        return 'Evening Rush Hour'
    else:
        return sunrise_sunset

us_acidents['PeriodOfDay'] = us_acidents.apply(lambda x: define_period_of_day(x['StartTime'], x['Civil_Twilight'], x['Nautical_Twilight'], x['Astronomical_Twilight'], x['Sunrise_Sunset']), axis=1)

In [61]:
localtime_dimension = us_acidents[['LocalTimeKey', 'StartTime', 'PeriodOfDay', 'Timezone', 'EndTime']]

### StandardTime Dimension

O *standard timezone* definido será o GMT - Greenwich Mean Time.

In [62]:
us_acidents["Timezone"].value_counts()

US/Eastern     20503
US/Pacific     13664
US/Central     11304
US/Mountain     2705
Name: Timezone, dtype: int64

In [63]:
# Creating the Standard Time Key
us_acidents['StandardTimeKey'] = pd.RangeIndex(start=1, stop=us_acidents.shape[0]+1)

In [64]:
import pytz

def convert_timezone(timestamp, old_timezone, new_timezone) :
    localized_timestamp = pytz.timezone(old_timezone).localize(timestamp)
    new_timezone_timestamp = localized_timestamp.astimezone(pytz.timezone(new_timezone))
    return dt.time(new_timezone_timestamp.hour, new_timezone_timestamp.minute, new_timezone_timestamp.second)

us_acidents['StandardStartTime'] = us_acidents.apply(lambda x: convert_timezone(x['Start_Datetime'], x['Timezone'], "Etc/GMT"), axis=1)
us_acidents['StandardEndTime'] = us_acidents.apply(lambda x: convert_timezone(x['End_Datetime'], x['Timezone'], "Etc/GMT"), axis=1)

In [65]:
us_acidents["Standard_Timezone"] = "GMT"
us_acidents[['StandardTimeKey','StartTime','EndTime','Timezone','StandardStartTime','StandardEndTime', 'Standard_Timezone']]
standardtime_dimension = us_acidents[['StandardTimeKey', 'StandardStartTime','StandardEndTime', 'Standard_Timezone']]

In [66]:
standardtime_dimension

,StandardTimeKey,StandardStartTime,StandardEndTime,Standard_Timezone
0,1,16:53:19,17:23:19,GMT
1,2,13:12:44,13:42:44,GMT
2,3,15:58:52,16:28:52,GMT
3,4,10:46:52,11:16:52,GMT
4,5,23:52:00,04:00:00,GMT
...,...,...,...,...
48171,48172,14:55:55,15:23:48,GMT
48172,48173,18:00:00,19:00:00,GMT
48173,48174,17:42:59,21:42:59,GMT
48174,48175,20:10:33,00:10:33,GMT


In [67]:
us_acidents.rename(columns={"Severity": "SeverityKey"}, inplace=True)

In [68]:
""" Per source, severity of the accident, a number between 1 and 4, where 1 indicates the least impact on traffic 
    (i.e., short delay as a result of the accident) and 4 indicates a significant impact on traffic (i.e., long delay).
"""
def correspond_severity(x):
    if x == 1 :
        return "Low impact - Short delay in traffic."
    elif x == 2:
        return "Moderate impact - Medium delay in traffic."
    elif x == 3:
        return "High impact - Considerable delay in traffic. Low risk of casualties."
    else:
        return "Significative impact - Long delay in traffic. High risk of casualties."
    
us_acidents["SeverityDescription"] = us_acidents["SeverityKey"].map(correspond_severity)

severity_dimension = us_acidents[["SeverityKey", "SeverityDescription"]]

severity_dimension.drop_duplicates(subset=['SeverityKey', 'SeverityDescription'], keep='last', inplace=True)
severity_dimension.sort_values(by='SeverityKey', inplace=True)
severity_dimension.reset_index(inplace=True)
severity_dimension.drop(columns=['index'], inplace=True)

C:\Users\valei\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\valei\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\valei\Anaconda3\lib\site-packages\pandas\core\frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


### Facts Table

In [69]:
us_acidents['Duration']=(us_acidents['End_Datetime']-us_acidents['Start_Datetime'])

In [70]:
facts_table = us_acidents[['ID', 'LocalTimeKey','StandardTimeKey', 'DateKey','LocationKey','WeatherKey','POI_Key','SeverityKey','TMC','Duration','Distance(mi)']]

------------------------

### Estrutura das Dimensões e Tabela de Factos

Nesta secção estão representadas as dimensões e o tipo de cada atributo das mesmas, além da tabela de factos. 

Estes formatos serão a base para o carregamento dos dados.

#### Dimensão 'Date'

In [71]:
Image(url= "https://docs.google.com/uc?export=download&id=1T8xO241Cxb6N9qczddVqihqQeibBo4WJ")

Esta dimensão vai ter a seguinte hierarquia:

In [72]:
Image(url= "https://docs.google.com/uc?export=download&id=1Qr_K-Dmd-2nm2Beem7lpZ4BzN0TTjWvQ")

In [73]:
date_dimension.head(5)

,DateKey,StartDay,DayOfWeek,WeekdayWeekend,WeekNumber,WorkDay,Holiday,SchoolBreak,StartMonth,Quarter,StartYear,EndDay,EndMonth,EndYear
0,1,8,Monday,Week Day,6,Work Day,Not Holiday,Classes,February,1,2016,8,February,2016
1,2,9,Tuesday,Week Day,6,Work Day,Not Holiday,Classes,February,1,2016,9,February,2016
2,3,16,Tuesday,Week Day,7,Work Day,Not Holiday,No Classes,February,1,2016,16,February,2016
3,4,18,Thursday,Week Day,7,Work Day,Not Holiday,No Classes,February,1,2016,18,February,2016
4,5,3,Thursday,Week Day,9,Work Day,Not Holiday,Classes,March,1,2016,3,March,2016


#### Dimensão 'LocalTime'

In [74]:
Image(url= "https://docs.google.com/uc?export=download&id=1-cUdrW8jV9qYzfkRjC0MwE1Ct20RRyji")

In [75]:
localtime_dimension.head(5)

,LocalTimeKey,StartTime,PeriodOfDay,Timezone,EndTime
0,1,11:53:19,Day,US/Eastern,12:23:19
1,2,08:12:44,Morning Rush Hour,US/Eastern,08:42:44
2,3,10:58:52,Day,US/Eastern,11:28:52
3,4,05:46:52,Night,US/Eastern,06:16:52
4,5,18:52:00,Night,US/Eastern,23:00:00


#### Dimensão 'StandardTime'

In [76]:
Image(url= "https://docs.google.com/uc?export=download&id=1CGd3_5Evw2o2731Q0lX3SMegLNJFO2BH")

In [77]:
standardtime_dimension.head(5)

,StandardTimeKey,StandardStartTime,StandardEndTime,Standard_Timezone
0,1,16:53:19,17:23:19,GMT
1,2,13:12:44,13:42:44,GMT
2,3,15:58:52,16:28:52,GMT
3,4,10:46:52,11:16:52,GMT
4,5,23:52:00,04:00:00,GMT


#### Dimensão 'Location'

In [78]:
Image(url= "https://docs.google.com/uc?export=download&id=1XNvJApisq0Ppjzwkxrskc4pbTeYWchXt")

Esta dimensão vai ter a seguinte hierarquia:

In [79]:
Image(url= "https://docs.google.com/uc?export=download&id=11aRI12E8_CUQ_JFL-VEixMn_7h-BhGzY")

In [80]:
location_dimension.head(5)

,LocationKey,Street,City,County,State,Subregion,Region,AlcoholConsumptionRate,NumberOfVehicleRegistrations,UrbanRuralClassification,MaximumSpeedLimit
0,1,Springboro Pike,Miamisburg,Montgomery,Ohio,East North Central,Midwest,(Medium) 2.0 < rate <= 2.5,(High) 10000000 < registrations <= 20000000,Small Metro,(Medium) 70 <= limit < 80
1,2,Clearcreek Rd,Springboro,Warren,Ohio,East North Central,Midwest,(Medium) 2.0 < rate <= 2.5,(High) 10000000 < registrations <= 20000000,Large Fringe Metro,(Medium) 70 <= limit < 80
2,3,Radio Rd,Dayton,Greene,Ohio,East North Central,Midwest,(Medium) 2.0 < rate <= 2.5,(High) 10000000 < registrations <= 20000000,Small Metro,(Medium) 70 <= limit < 80
3,4,Little York Rd,Dayton,Montgomery,Ohio,East North Central,Midwest,(Medium) 2.0 < rate <= 2.5,(High) 10000000 < registrations <= 20000000,Small Metro,(Medium) 70 <= limit < 80
4,5,I-75 N,Dayton,Montgomery,Ohio,East North Central,Midwest,(Medium) 2.0 < rate <= 2.5,(High) 10000000 < registrations <= 20000000,Small Metro,(Medium) 70 <= limit < 80


#### Dimensão 'Weather'

In [81]:
Image(url= "https://docs.google.com/uc?export=download&id=17ojEjQMr-QYjP9LeQHM6ZaAwHuY-2QTS")

In [82]:
weather_dimension.head(5)

,WeatherKey,Temperature(F),Visibility(mi),Humidity(%),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition
0,1,(Low) 32.0 < temp <= 50.0,(Good) 5.0 < visibility <= 20.0,(Very High) rate > 75.0,WSW,(Gentle Breeze) 7.38 < speed <= 12.30,(Light) rate <= 0.1,Overcast
1,2,(Cold) temp <= 32.0,(Reduced) 1.0 < visibility <= 2.0,(Very High) rate > 75.0,WSW,(Gentle Breeze) 7.38 < speed <= 12.30,(Light) rate <= 0.1,Light Snow
2,3,(Cold) temp <= 32.0,(Good) 5.0 < visibility <= 20.0,(Very High) rate > 75.0,WNW,(Light Breeze) 3.36 < speed <= 7.38,Unknown,Overcast
3,4,(Cold) temp <= 32.0,(Good) 5.0 < visibility <= 20.0,(Very High) rate > 75.0,Calm,Unknown,Unknown,Mostly Cloudy
4,5,(Cold) temp <= 32.0,(Reduced) 1.0 < visibility <= 2.0,(Very High) rate > 75.0,SE,(Light Breeze) 3.36 < speed <= 7.38,(Light) rate <= 0.1,Light Snow


#### Dimensão 'POI'

In [83]:
Image(url= "https://docs.google.com/uc?export=download&id=1g1sa8oHNx0Aeh9_oaAT1kSzr9v7rdjef")

In [84]:
poi_dimension.head(5)

,POI_Key,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop
0,1,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present
1,2,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present
2,3,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present
3,4,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Present,Not Present
4,5,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present


#### Dimensão 'TMC'

In [85]:
Image(url= "https://docs.google.com/uc?export=download&id=1t5qsDQ-a2cQA16i3h475mGRjhWluiAdz")

In [86]:
event_code.head(5)

,TMC_Key,Event
0,1,traffic problem
1,2,queuing traffic (with average speeds Q). Dange...
2,11,overheight warning system triggered
3,12,"(Q) accident(s), traffic being directed around..."
4,16,"closed, rescue and recovery work in progress"


#### Dimensão 'Severity'

In [87]:
Image(url= "https://docs.google.com/uc?export=download&id=1qOjq5Ix50WDqTE_cnEEdYj-wq-hwA0Us")

In [88]:
severity_dimension

,SeverityKey,SeverityDescription
0,1,Low impact - Short delay in traffic.
1,2,Moderate impact - Medium delay in traffic.
2,3,High impact - Considerable delay in traffic. L...
3,4,Significative impact - Long delay in traffic. ...


### Tabela de Factos

A inserção de um registo na tabela de factos requer a inserção dos dados nas restantes dimensões *a priori*, de modo a poder obter as *surrogate keys* como chaves estrangeiras para a tabela.

In [89]:
Image(url= "https://docs.google.com/uc?export=download&id=1JG4ZcPS3lcrG_Jr4AOS1XJMyUVneP2AD")

In [90]:
facts_table.head(5)

,ID,LocalTimeKey,StandardTimeKey,DateKey,LocationKey,WeatherKey,POI_Key,SeverityKey,TMC,Duration,Distance(mi)
0,A-23,1,1,1,1,1,1,2,201,00:30:00,0.01
1,A-64,2,2,2,2,2,2,2,201,00:30:00,0.01
2,A-197,3,3,3,3,3,3,2,201,00:30:00,0.00
3,A-235,4,4,4,4,4,4,2,201,00:30:00,0.01
4,A-477,5,5,5,5,5,5,3,201,04:08:00,0.01


### Carregamento dos Dados

In [91]:
Image(url= "https://docs.google.com/uc?export=download&id=1QRTZGzG2LOoIUaEPKKIKHL5-hJLyUZ1W", width=800, height=650)

In [92]:
import psycopg2 as pg

#### Criação de tabelas

In [93]:
create_statements_list = []
drop_statements_list = []

create_local_time_table = ('CREATE TABLE local_time_dimension', """create table local_time_dimension(
                                local_time_key int,
                                start_time time,
                                period_of_day char(32), 
                                timezone char(16),
                                end_time time,
                                primary key (local_time_key)
                            )""")
create_statements_list.append(create_local_time_table)

create_standard_time_table = ('CREATE TABLE standard_time_dimension', """create table standard_time_dimension(
                                    standard_time_key int,
                                    start_time time,
                                    standard_timezone char(16),
                                    end_time time,
                                    primary key (standard_time_key)
                            )""")
create_statements_list.append(create_standard_time_table)

create_location_table = ('CREATE TABLE location_dimension', """create table location_dimension(
                                location_key int, 	
                                street char(64), 	
                                city char(64), 	
                                county char(64),	
                                state char(64),	
                                subregion char(32), 
                                region char(32), 	
                                alcohol_consumption_rate char(32),	
                                number_of_vehicle_registrations char(64),	
                                urban_rural_classification char(32),	
                                maximum_speed_limit char(32),
                                primary key (location_key)
                            )""")

create_statements_list.append(create_location_table)
                            
create_date_table = ('CREATE TABLE date_dimension', """create table date_dimension(
                                    date_key int,
                                    start_day int,
                                    day_of_week char(16),
                                    weekday_or_weekend char(8),
                                    week_number int,
                                    work_day char(16),
                                    holiday char(64),
                                    school_break char(16),
                                    start_month char(16),
                                    quarter int,
                                    start_year int,
                                    end_day int,
                                    end_month char(16),
                                    end_year int,
                                    primary key (date_key)
                            )""")
create_statements_list.append(create_date_table)

create_poi_table = ('CREATE TABLE poi_dimension', """create table poi_dimension(
                                    poi_key int,
                                    presence_of_amenity char(16),
                                    presence_of_bump char(16),
                                    presence_of_crossing char(16),
                                    presence_of_giveway char(16),
                                    presence_of_junction char(16),
                                    presence_of_noexit char(16),
                                    presence_of_rail_way char(16),
                                    presence_of_roundabout char(16),
                                    presence_of_station char(16),
                                    presence_of_stop char(16),
                                    presence_of_traffic_calming char(16),
                                    presence_of_traffic_signal char(16),
                                    presence_of_turning_loop char(16),
                                    primary key (poi_key)
                            )""")
create_statements_list.append(create_poi_table)
                            
create_tmc_table = ('CREATE TABLE tmc_dimension', """create table tmc_dimension(
                            tmc_key int,
                            event char(256),
                            primary key (tmc_key)  
                    )
                    """)
create_statements_list.append(create_tmc_table)

create_weather_table = ('CREATE TABLE weather_dimension', """create table weather_dimension(
                                weather_key int,
                                temperature char(128),
                                visibility char(128),
                                humidity char(128),
                                wind_direction char(128),
                                wind_speed char(128),
                                precipitation char(128),
                                weather_condition char(128),
                                primary key (weather_key)
                   )
                   """)
create_statements_list.append(create_weather_table)

create_severity_table = ('CREATE TABLE severity_dimension', """create table severity_dimension(
                                severity_key int,
                                description char(128),
                                primary key (severity_key)
                   )
                   """)
create_statements_list.append(create_severity_table)

create_facts_table = ('CREATE TABLE facts_table', """create table facts_table(
                                accidentid varchar(32),
                                local_timekey int,
                                standard_timekey int,
                                datekey int,
                                locationkey int,
                                weatherkey int,
                                poikey int,
                                severitykey int,
                                tmckey int,
                                duration interval,
                                distance float,
                                primary key (accidentid),
                                foreign key (local_timekey) references local_time_dimension (local_time_key),
                                foreign key (standard_timekey) references standard_time_dimension (standard_time_key),
                                foreign key (datekey) references date_dimension (date_key),
                                foreign key (locationkey) references location_dimension (location_key),
                                foreign key (weatherkey) references weather_dimension (weather_key),
                                foreign key (poikey) references poi_dimension (poi_key),
                                foreign key (severitykey) references severity_dimension (severity_key),
                                foreign key (tmckey) references tmc_dimension (tmc_key)
                )
                """)
create_statements_list.append(create_facts_table)

# DROP FACTS TABLE STATEMENT
drop_facts_table = ('DROP TABLE facts_table', "DROP TABLE IF EXISTS facts_table CASCADE")
drop_statements_list.append(drop_facts_table)

# DROP TABLE STATEMENTS
drop_local_time_table = ('DROP TABLE local_time_dimension', "DROP TABLE IF EXISTS local_time_dimension")
drop_statements_list.append(drop_local_time_table)
drop_standard_time_table = ('DROP TABLE standard_time_dimension', "DROP TABLE IF EXISTS standard_time_dimension")
drop_statements_list.append(drop_standard_time_table)
drop_location_table = ('DROP TABLE location_dimension', "DROP TABLE IF EXISTS location_dimension")
drop_statements_list.append(drop_location_table)
drop_date_table = ('DROP TABLE date_dimension', "DROP TABLE IF EXISTS date_dimension")
drop_statements_list.append(drop_date_table)
drop_poi_table = ('DROP TABLE poi_dimension', "DROP TABLE IF EXISTS poi_dimension")
drop_statements_list.append(drop_poi_table)
drop_weather_table = ('DROP TABLE weather_dimension', "DROP TABLE IF EXISTS weather_dimension")
drop_statements_list.append(drop_weather_table)
drop_severity_table = ('DROP TABLE severity_dimension', "DROP TABLE IF EXISTS severity_dimension")
drop_statements_list.append(drop_severity_table)
drop_tmc_table = ('DROP TABLE tmc_dimension', "DROP TABLE IF EXISTS tmc_dimension")
drop_statements_list.append(drop_tmc_table)

In [95]:
HOST = "10.101.151.25"
DATABASE = "tpd017"
USER = "tpd017"
PASSWORD = "tpd017" 

# Given a statement list, execute all the statements inside.
# Recommended either to build or drop a database.
def execute_multiple_statements(statements):
    conn = pg.connect(host=HOST, database=DATABASE, user=USER, password=PASSWORD)
    cur = conn.cursor()
    print('Executing multiple statements:')
    for statement, sql in statements:
        print('\t%s' % statement)
        cur.execute(sql)
    print('\nFinished!')
    cur.close()
    conn.commit()
    conn.close()

# Execute a single statement.
def execute_single_statement(statement):
    conn = pg.connect(host=HOST, database=DATABASE, user=USER, password=PASSWORD)
    cur = conn.cursor()
    print('Executing %s ...' % statement[0])
    cur.execute(statement[1])
    print('\nFinished!')
    cur.close()
    conn.commit()
    conn.close()

# Call for building a database.
def build_database():
    return execute_multiple_statements(create_statements_list)

# Call for dropping a database.
def drop_database():
    return execute_multiple_statements(drop_statements_list)

# Requires connection to the VPN
drop_database()
build_database()

Executing multiple statements:
	DROP TABLE facts_table
	DROP TABLE local_time_dimension
	DROP TABLE standard_time_dimension
	DROP TABLE location_dimension
	DROP TABLE date_dimension
	DROP TABLE poi_dimension
	DROP TABLE weather_dimension
	DROP TABLE severity_dimension
	DROP TABLE tmc_dimension

Finished!
Executing multiple statements:
	CREATE TABLE local_time_dimension
	CREATE TABLE standard_time_dimension
	CREATE TABLE location_dimension
	CREATE TABLE date_dimension
	CREATE TABLE poi_dimension
	CREATE TABLE tmc_dimension
	CREATE TABLE weather_dimension
	CREATE TABLE severity_dimension
	CREATE TABLE facts_table

Finished!


#### Carregamento dos dados pelas tabelas


#### Dimensão 'Date'

In [108]:
conn = pg.connect(host=HOST,database=DATABASE, user=USER, password=PASSWORD)
curs = conn.cursor()
print("Populating Date Dimension...")
for row in list(date_dimension.itertuples(index=False, name=None)):
    datekey,std,stm,sty,endd,endm,endy,dow,wkwkend,qtr,wknum,holi,workd,schbrk = row
    curs.execute("insert into date_dimension values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", 
        (datekey,std,stm,sty,endd,endm,endy,dow,wkwkend,qtr,wknum,holi,workd,schbrk))
curs.close()
conn.commit()
conn.close()
print("Success!")

Populating Date Dimension...
Success!



#### Dimensão 'LocalTime'

In [109]:
conn = pg.connect(host=HOST,database=DATABASE, user=USER, password=PASSWORD)
curs = conn.cursor()
print("Populating LocalTime Dimension...")
for row in list(localtime_dimension.itertuples(index=False, name=None)):
    localkey,st,prd,tz,end = row
    curs.execute("insert into local_time_dimension values(%s, %s, %s, %s, %s)", (localkey,st,prd,tz,end))
curs.close()
conn.commit()
conn.close()
print("Success!")

Populating LocalTime Dimension...
Success!


#### Dimensão 'StandardTime'

In [110]:
conn = pg.connect(host=HOST,database=DATABASE, user=USER, password=PASSWORD)
curs = conn.cursor()
print("Populating StandardTime Dimension...")
for row in list(standardtime_dimension.itertuples(index=False, name=None)):
    standardkey, standardst, standardend, standardtz = row
    curs.execute("insert into standard_time_dimension values(%s, %s, %s, %s)", 
    (standardkey, standardst, standardtz, standardend))
curs.close()
conn.commit()
conn.close()
print("Success!")

Populating StandardTime Dimension...
Success!


#### Dimensão 'Location'

In [111]:
conn = pg.connect(host=HOST,database=DATABASE, user=USER, password=PASSWORD)
curs = conn.cursor()
print("Populating Location Dimension...")
for row in list(location_dimension.itertuples(index=False, name=None)):
    locationkey,street,city,county,state,reg,subreg,alc,veh,urc,spd = row
    curs.execute("insert into location_dimension values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", 
        (locationkey,street,city,county,state,reg,subreg,alc,veh,urc,spd))
curs.close()
conn.commit()
conn.close()
print("Success!")

Populating Location Dimension...
Success!


#### Dimensão 'Severity'

In [112]:
conn = pg.connect(host=HOST,database=DATABASE, user=USER, password=PASSWORD)
curs = conn.cursor()
print("Populating Severity Dimension...")
for row in list(severity_dimension.itertuples(index=False, name=None)):
    sevkey,event = row
    curs.execute("insert into severity_dimension values(%s, %s)", (sevkey, event))
curs.close()
conn.commit()
conn.close()
print("Success!")

Populating Severity Dimension...
Success!


#### Dimensão 'Weather'

In [113]:
conn = pg.connect(host=HOST,database=DATABASE, user=USER, password=PASSWORD)
curs = conn.cursor()
print("Populating Weather Dimension...")
for row in list(weather_dimension.itertuples(index=False, name=None)):
    weatherkey,temp,vis,hum,windir,winsp,prec,cond = row
    curs.execute("insert into weather_dimension values(%s, %s, %s, %s, %s, %s, %s, %s)", 
    (weatherkey,temp,vis,hum,windir,winsp,prec,cond))
curs.close()
conn.commit()
conn.close()
print("Success!")

Populating Weather Dimension...
Success!


#### Dimensão 'POI'

In [114]:
conn = pg.connect(host=HOST,database=DATABASE, user=USER, password=PASSWORD)
curs = conn.cursor()
print("Populating POI Dimension...")
for row in list(poi_dimension.itertuples(index=False, name=None)):
    poikey,ame,bump,cross,givw,junc,nox,rw,rnd,st,stop,calm,sig,loop = row
    curs.execute("insert into poi_dimension values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", 
    (poikey,ame,bump,cross,givw,junc,nox,rw,rnd,st,stop,calm,sig,loop))
curs.close()
conn.commit()
conn.close()
print("Success!")

Populating POI Dimension...
Success!


#### Dimensão 'TMC'

In [115]:
conn = pg.connect(host=HOST,database=DATABASE, user=USER, password=PASSWORD)
curs = conn.cursor()
print("Populating TMC Dimension...")
for row in list(event_code.itertuples(index=False, name=None)):
    tmckey, event = row
    curs.execute("insert into tmc_dimension values(%s, %s)", (tmckey, event))
curs.close()
conn.commit()
conn.close()
print("Success!")

Populating TMC Dimension...
Success!


#### Facts Table

In [116]:
conn = pg.connect(host=HOST,database=DATABASE, user=USER, password=PASSWORD)
curs = conn.cursor()
print("Populating Facts' Table...")
for row in list(facts_table.itertuples(index=False, name=None)):
    id,ltk,stk,dtk,lck,wtrk,poik,svrk,tmck,dur,dist = row
    curs.execute("insert into facts_table values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", 
    (id,ltk,stk,dtk,lck,wtrk,poik,svrk,tmck,dur,dist))
curs.close()
conn.commit()
conn.close()
print("Success!")

Populating Facts' Table...
Success!


## Armazenamento e tamanho dos dados

Esta secção procura fazer uma descrição do tamanho que os dados irão ocupar na base de dados. Como é possível ver, dado que muitas dimensões partilham o mesmo número de linhas que a tabela de factos, estas acabam por levar a um grande crescimento do tamanho da base de dados.

No que importa em relação ao crescimento no número de linhas, a inserção de um registo leva ao crescimento das tabelas *Date*, *LocalTime*, *StandardTime*, *Location*, *Weather*, *POI* e da tabela de factos. Apenas as tabelas *TMC* e *Severity* se mantêm invariáveis.

Para uma *sample* de 50000 registos, o tamanho que a BD ocupa encontra-se por volta de 155 MB.
Em estimativa, observando o número total de registos (pouco menos de três milhões), o tamanho que o *data warehouse* ocuparia seria de ~9 GB.

In [105]:
usage_dict = {"Dimension":[], "Usage (byte)":[]}

usage_dict["Dimension"].append("Date")
usage_dict["Usage (byte)"].append(date_dimension.memory_usage(index=False,deep=True).sum())

usage_dict["Dimension"].append("LocalTime")
usage_dict["Usage (byte)"].append(localtime_dimension.memory_usage(index=False,deep=True).sum())

usage_dict["Dimension"].append("StandardTime")
usage_dict["Usage (byte)"].append(standardtime_dimension.memory_usage(index=False,deep=True).sum())

usage_dict["Dimension"].append("Location")
usage_dict["Usage (byte)"].append(location_dimension.memory_usage(index=False,deep=True).sum())

usage_dict["Dimension"].append("Weather")
usage_dict["Usage (byte)"].append(weather_dimension.memory_usage(index=False,deep=True).sum())

usage_dict["Dimension"].append("Severity")
usage_dict["Usage (byte)"].append(severity_dimension.memory_usage(index=False,deep=True).sum())

usage_dict["Dimension"].append("POI")
usage_dict["Usage (byte)"].append(poi_dimension.memory_usage(index=False,deep=True).sum())

usage_dict["Dimension"].append("TMC")
usage_dict["Usage (byte)"].append(event_code.memory_usage(index=False,deep=True).sum())

usage_dict["Dimension"].append("Facts Table")
usage_dict["Usage (byte)"].append(facts_table.memory_usage(index=False,deep=True).sum())

val = sum(usage_dict["Usage (byte)"])
usage_dict["Dimension"].append("Total")
usage_dict["Usage (byte)"].append(val)

pd.DataFrame(usage_dict)


,Dimension,Usage (byte)
0,Date,24382349
1,LocalTime,11377416
2,StandardTime,7900864
3,Location,35179574
4,Weather,27563154
5,Severity,476
6,POI,42707072
7,TMC,155471
8,Facts Table,6052019
9,Total,155318395


### Análise do Processo ETL

* *Describe potential issues with the ETL procedure used*

    1. Um problema que encontramos logo à partida refere-se ao tamanho dos dados. Não há poder de computação suficiente para conseguir carregar o *data warehouse* inteiro, o que implica que trabalhemos com uma sample. Consequente, as interrogações OLAP feitas podem não corresponder à realidade descrita pelos dados;

    2. Muitas das dimensões criadas têm o mesmo número de linhas da tabela de factos, o que implica que o seu carregamento seja mais longo e que posteriormente o processo de carregamento de novos dados seja vulnerável a erros.


* *Compare your schema to the one previously defined in phase I*

    1. Transformação da medida *Severity* numa dimensão, dado que a medida por si só não iria conseguir ser alvo de quaisquer operações artiméticas;
    2. Remoção do atributo *Moon Calendar*, por não se considerar que seria relevante para os processos de negócio que envolvem o *warehouse*;
    3. Adição de atributos relativamente à região e subregião da localização do acidente, de forma a estender a hierarquia entre os dados da dimensão.
    4. Além das transformações de dados indicadas na Fase I, alguns atributos foram renomeados ou sofreram alterações no seu tipo para se tornarem mais explicítos quando se trata de ações de *decision making*.
    

* *Discuss the issues for updating the data warehouse with novel data*

    1.  O crescimento da BD pode vir a ser elevado dado que a colocação de um registo implica o crescimento de várias dimensões. Apenas a TMC e a Severity são estáticas.
    2.  Na DBMS, muitos dos atributos não têm restrições de domínio, o que permite que ao carregar novos dados, seja possível colocar valores nesses atributos que não correspondam a qualquer categoria instituída no processo ETL.
    3. Também será necessário, aquando de nova inserção, verificar quais as últimas surrogate keys por cada dimensão.
   

### Descrição do *Star Schema*

O *Star Schema* do *data warehouse* corresponde a um datamart único, dividindo entre a tabela de factos e oito dimensões.

Em termos do *grain* da tabela de factos, cada registo corresponde **a um acidente com um identificador único, que ocorreu em determinada data, em certa hora local e hora standard, em dada localização com certos pontos de interesse, com várias condições meterológicas, tendo uma determinada severidade e um evento associado. O acidente teve determinada duração e ocorreu ao longo de uma certa distância.**

In [106]:
Image(url= "https://docs.google.com/uc?export=download&id=1CyzIAFLXLTOve3jIHUfo_rgQF08enB11", width=750, height=650)

#### *Bus Matrix*

Dado que não mostramos na Fase I os diferentes processos de negócio que poderiam constituir o *data warehouse*, anexamos então a Bus Matrix com as alterações feitas nas dimensões.

É possível identificar que as dimensões relativamente à data do acidente, tempo em que ocorreu (local) e condições metereológicas correspondem àquelas que são de maior interesse.

In [107]:
Image(url= "https://docs.google.com/uc?export=download&id=1eRAjATWxGoP2lFuQMa5WlUVjzngoyv-s")

## Fontes dos *datasets* externos

1. Moosavi, S. (2020), A Countrywide Traffic Accident Dataset (2016 - 2019), Kaggle,
https://www.kaggle.com/sobhanmoosavi/us-accidents;
2. Moosavi, S. (2019), A Countrywide Traffic Accident Dataset (2016 - 2019),
https://smoosavi.org/datasets/us_accidents;
3. CDC/National Center for Health Statistics (2017), NCHS Urban-Rural Classification Scheme for
Counties, https://www.cdc.gov/nchs/data_access/urban_rural.htm;
4. OpenStreetMap (2020), TMC - Event Code List,
https://wiki.openstreetmap.org/wiki/TMC/Event_Code_List;
5. Wikipedia (2020), Speed limits in the United States,
https://en.wikipedia.org/wiki/Speed_limits_in_the_United_States#Overview;
6. Elflein, J. (2019), Per capita alcohol consumption of all beverages in the U.S. by state - 2017,
https://www.statista.com/statistics/442848/per-capita-alcohol-consumption-of-all-beverages-in-the-us-by
-state/;
7. U.S. Office of Highway Policy Information (2018), State Motor-Vehicle Registrations - 2018,
https://www.fhwa.dot.gov/policyinformation/statistics/2018/mv1.cfm;
8. List of USA Federal Holiday
